## Парсинг новостей

Скорее всего, новости, оказавшие наибольшее влияние на потребительский спрос, находятся на наиболее популярных новостных ресурсах.

Рейтинг новостных ресурсов по посещаемости - лето 2020 г.
<https://infoselection.ru/infokatalog/novosti-smi/smi/item/249-20-samykh-poseshchaemykh-novostnykh-resursov-runeta>

Для парсинга я проверил топ 5 по посещаемости:
1. Комсомольская правда - парсинг возможен, через запрос к их cdn можно получать страницы новостей по датам, но сайт сильно разделен по категориям, что вызывает сложности, пока не делал, если нужно будет, сделаю.
2. РИА Новости - всё ок.
3. Lenta.ru - всё ок.
4. РосБизнесКонсалтинг - архив онлайн газеты не доступен для 2020 года.
5. Московский Комсомолец (МК) - имеет архив по датам, но при обращении выдает ошибку 404.

Поэтому парсинг я осуществляю для двух популярных новостных площадок - РИА Новости (52581 новость) и Lenta.ru (19940 новостей).

In [100]:
import requests
import datetime
import dateparser
import pandas as pd

from pathlib import Path
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from typing import Callable
from time import sleep


start_date = datetime.date(2020, 2, 1)
end_date   = datetime.date(2020, 5, 1) #не включительно

session = requests.Session()
session.headers.update({'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) '
                                      'AppleWebKit/537.36 (KHTML, like Gecko) '
                                      'Chrome/104.0.5112.102 Safari/537.36'})

In [29]:
def get_news_list_by_date_ria(date: datetime.date):
    date = date.strftime('%Y%m%d')
    
    page = 1
    news_list = []
        
    while True:
        r = session.get(f'https://ria.ru/{date}/', params={'page': page})
        
        if r.status_code == 404:
            break
        if not r.ok:
            continue
            
        soup = BeautifulSoup(r.text, 'html.parser')
        for item in soup.find("div", class_="list").children:
            if item.name != 'div':
                continue
            
            data = {}
            get_text = lambda res: res.text if res else None
            data['name']  = get_text(item.find(class_='list-item__content'))
            data['date']  = get_text(item.find(class_='list-item__date'))
            data['date']  = str(dateparser.parse(data['date'])) if data['date'] else None
            data['views'] = get_text(item.find(class_='list-item__views-text'))
            data['tags']  = list(map(get_text, item.find_all(class_='list-tag__text')))
            data['link']  = item.find('a')['href']
        
            news_list.append(data)
        page += 1
        
    return news_list

In [ ]:
def get_news_list_by_date_lenta(date: datetime.date):
    date = date.strftime('%Y/%m/%d')
    
    page = 1
    news_list = []
        
    while True:
        r = session.get(f'https://lenta.ru/{date}/page/{page}/')
        
        if not r.ok:
            continue
            
        soup = BeautifulSoup(r.text, 'html.parser')
        items = soup.find_all("li", class_="archive-page__item _news")
        
        if not items:
            break
        
        for item in items:
            data = {}
            get_text = lambda res: res.text if res else None
            data['name']  = get_text(item.find('h3'))
            data['date']  = get_text(item.find('time'))
            data['date']  = str(dateparser.parse(data['date'])) if data['date'] else None
            data['link']  = item.find('a')['href']
            data['link']  = ('https://lenta.ru' if data['link'][0] == '/' else '') + data['link']
        
            news_list.append(data)
            
        page += 1
        
    return news_list

In [20]:
def get_news_list(get_by_date: Callable, 
                  start_date: datetime.date, 
                  end_date: datetime.date,
                  delta=datetime.timedelta(days=1)):
    date = start_date
    news_list = []
    
    for _ in tqdm(range((end_date - start_date).days), desc="Days: "):
        news_list += get_by_date(date)
        date += delta
        
    return news_list

In [ ]:
def parse_news_list(name: str,
                    path: str,
                    get_by_date: Callable,
                    start_date: datetime.date = start_date, 
                    end_date: datetime.date = end_date):
    print(f'Getting {name} news by days:')
    data = get_news_list(get_by_date, start_date, end_date)
    print('Writing into csv')
    df = pd.DataFrame(data)
    df.to_csv(path)
    print(f'Done: {len(df)} news')

In [30]:
parse_news_list(name='RIA',
                path='parsed_ria_news_list.csv',
                get_by_date=get_news_list_by_date_ria)

Getting RIA news by days:


Days:   0%|          | 0/90 [00:00<?, ?it/s]

Writing into csv
Done


In [ ]:
parse_news_list(name='Lenta.ru',
                path='parsed_lenta_news_list.csv',
                get_by_date=get_news_list_by_date_lenta)

In [121]:
def extract_ria_text(soup: BeautifulSoup):
    article = soup.find('div', class_='article__body')
    if not article:
        return ''
    
    text = ''
    for item in article.children:
        if item.name == 'div' and 'data-type' in item.attrs and item['data-type'] == 'text':
            text += item.text + '\n'
    
    return text

In [127]:
def extract_lenta_text(soup: BeautifulSoup):
    items = soup.find_all('p', class_='topic-body__content-text')
    text = ''
    
    for item in items:
        text += item.text + '\n'
    
    return text

In [118]:
def get_text(link: str, extract_text: Callable, attempt: int=0, max_attempt: int=5):
    if attempt >= max_attempt:
        return ''
    
    r = session.get(link)
    if not r.ok:
        sleep(1)
        return get_text(link, extract_text, attempt + 1, max_attempt)

    text = extract_text(BeautifulSoup(r.text, 'html.parser'))
    if not text:
        return get_text(link, extract_text, attempt + 1, max_attempt)
    
    return text

In [119]:
def parse_texts(name: str, 
                path: str, 
                folder: str,
                extract_text: Callable,
                max_attempt=5):
    print(f'Getting {name} news texts:')
    df = pd.read_csv(path, index_col=0)
    
    folder = Path(folder)
    if not folder.exists():
        folder.mkdir()
    
    for row in tqdm(df.itertuples(), total=len(df), desc='News: '):
        filename = folder / (str(row.Index) + '.txt')
        if filename.exists():
            continue
        
        text = get_text(row.link, extract_text)
        if text:  
            with open(filename, 'w', encoding='utf-8') as file:
                file.write(text)
    print(f'Done {len(df)} news')

In [137]:
while True:
    try:
        parse_texts(name='RIA', 
            path='parsed_ria_news_list.csv', 
            folder='parsed_ria_texts',
            extract_text=extract_ria_text)
    except Exception:
        continue
    break

Getting RIA news texts:


News:   0%|          | 0/52581 [00:00<?, ?it/s]

Done 52581 news


In [ ]:
parse_texts(name='Lenta.ru', 
            path='parsed_lenta_news_list.csv', 
            folder='parsed_lenta_texts',
            extract_text=extract_lenta_text)

In [ ]:
def unite_datasets(path_folder: dict, path: str, columns: list):
    df = None
    for csv_path in path_folder:
        df_cur = pd.read_csv(csv_path, index_col=0)[columns]
        texts = [''] * len(df_cur)
        
        
        for file in Path(path_folder[csv_path]).iterdir():
            name = file.name
            if not name.endswith('.txt'):
                continue
                
            try:
                ind = int(name[:-4])
            except Exception:
                continue
            
            with open(file, encoding='utf-8') as f:
                texts[ind] = f.read()
                
        df_cur['text'] = pd.Series(texts, index=df_cur.index)
        
        if df is None:
            df = df_cur
        else:
            df = pd.concat([df, df_cur], ignore_index=True)
            
    if df is not None:
        df.to_csv(path)
    return df

In [146]:
df = unite_datasets({
    'parsed_ria_news_list.csv': 'parsed_ria_texts',
    'parsed_lenta_news_list.csv': 'parsed_lenta_texts'
}, 'parsed_raw_texts.csv', ['name', 'date', 'link'])
df.head()

,name,date,link,text
0,"Театральные премьеры февраля: ""Садко"", ""Чайка""...",2020-02-01 23:46:00,https://ria.ru/20200201/1564150494.html,"МОСКВА, 1 фев – РИА Новости, Наталия Курова. В..."
1,В США прооперировали архиепископа Кипрского,2020-02-01 23:39:00,https://ria.ru/20200201/1564152142.html,"АФИНЫ, 1 фев - РИА Новости. Архиепископ Кипрск..."
2,В Госдуме прокомментировали идею об упоминании...,2020-02-01 23:36:00,https://ria.ru/20200201/1564152109.html,"МОСКВА, 1 фев - РИА Новости. Первый зампред ко..."
3,"В Киеве рассчитывают, что местные выборы на Ук...",2020-02-01 23:27:00,https://ria.ru/20200201/1564152051.html,"КИЕВ, 1 фев - РИА Новости. Помощник президента..."
4,Клишас оценил предложение патриарха об упомина...,2020-02-01 23:26:00,https://ria.ru/20200201/1564152029.html,"МОСКВА, 1 фев - РИА Новости. Предложение об уп..."
